In [ ]:
import re
from pprint import pprint
from data import sentences

In [16]:
def extract_entities(tokens, labels):
    """
    Agrupa tokens contíguos que compartilham o mesmo tipo de entidade IOB.
    Retorna um dicionário: { 'VALOR': [ '500 mil reais', ... ], 'ATIVO': [ 'PETR4', ... ] }
    """
    entities = {}
    current_tokens = []
    current_type = None

    for tok, lab in zip(tokens, labels):
        if lab.startswith("B-"):
            # Se já havia uma entidade sendo construída, salva-a antes
            if current_type is not None:
                entities.setdefault(current_type, []).append(" ".join(current_tokens))
            current_type = lab.split("-", 1)[1]  # Ex: de 'B-VALOR' extrai 'VALOR'
            current_tokens = [tok]
        elif lab.startswith("I-") and current_type == lab.split("-", 1)[1]:
            # Continua a mesma entidade
            current_tokens.append(tok)
        else:
            # Encerramento da entidade anterior, se houver
            if current_type is not None:
                entities.setdefault(current_type, []).append(" ".join(current_tokens))
                current_type = None
                current_tokens = []
            # Se for 'O', simplesmente ignora

    # Se a última entidade não foi salva dentro do loop, salva-a agora
    if current_type is not None:
        entities.setdefault(current_type, []).append(" ".join(current_tokens))

    return entities

In [ ]:
# --------------------------------------------------
# 2) Utilitários
# --------------------------------------------------
money_units = {"reais", "mil", "milhão", "milhões", "bilhão", "bilhões"}
currency_symbols = {"R$", "$", "US$"}
invest_verbs = {"investir", "aplicar", "aport", "destinar", "colocar", "comprar", "reservar"}

ticker_re = re.compile(r"^[A-Z]{4}\d{1,2}$")
number_re = re.compile(r"^\d[\d.,]*$")             # 123  1.234,56
simple_tokenizer = re.compile(r"\w+|\S")           # palavras e pontuação separadas

def tokenize(sent):
    return simple_tokenizer.findall(sent)

def is_value_token(tok):
    """Heurística: símbolo de moeda OU número (com , .)"""
    return (tok in currency_symbols) or bool(number_re.fullmatch(tok))

def is_money_unit(tok):
    return tok.lower() in money_units

def is_ticker(tok):
    return bool(ticker_re.fullmatch(tok))

def label_tokens(tokens):
    """
    Retorna lista de rótulos IOB (B-VALOR/I-VALOR/B-ATIVO/I-ATIVO/O)
    baseada em heurísticas simples.
    """
    labels = ["O"] * len(tokens)
    i = 0
    while i < len(tokens):
        tok = tokens[i]

        # ---------------- VALOR ----------------
        if is_value_token(tok):
            labels[i] = "B-VALOR"
            j = i + 1
            # captura "mil", "reais" etc. imediatamente depois
            while j < len(tokens) and (is_value_token(tokens[j]) or is_money_unit(tokens[j])):
                labels[j] = "I-VALOR"
                j += 1
            i = j
            continue

        # ---------------- ATIVO ----------------
        # marca tickers ou "Tesouro Selic 2028" / "FII XYZ11" / "ETF ABCD11"
        if is_ticker(tok):
            labels[i] = "B-ATIVO"
            i += 1
            continue

        # caso especial: tokens 'FII' ou 'ETF' ou 'Tesouro' iniciando ativo multi-token
        if tok in {"FII", "ETF", "Tesouro"}:
            labels[i] = "B-ATIVO"
            j = i + 1
            # marca sequência até encontrar pontuação ou verbo de ação ou fim
            while j < len(tokens) and not re.fullmatch(r"[.,]", tokens[j]):
                labels[j] = "I-ATIVO"
                # para se bater num verbo-de-ação (investe, comprar...) ou preposição sem sentido
                if tokens[j].lower().split(" ")[0] in invest_verbs:
                    break
                j += 1
            i = j
            continue

        i += 1
    return labels

def token2features(sent_tokens, i):
    """Cria dicionário de features simples para o token i em sent_tokens."""
    tok = sent_tokens[i]
    prev_tok = sent_tokens[i-1] if i > 0 else "__BOS__"
    next_tok = sent_tokens[i+1] if i < len(sent_tokens)-1 else "__EOS__"

    return {
        "bias": 1.0,
        "token": tok,
        "lower": tok.lower(),
        "is_upper": tok.isupper(),
        "is_digit": tok.isdigit(),
        "has_digit": any(ch.isdigit() for ch in tok),
        "suffix2": tok[-2:],
        "prev_token": prev_tok,
        "next_token": next_tok,
        "prev_is_currency": prev_tok in currency_symbols,
        "next_is_currency": next_tok in currency_symbols,
        "is_ticker": bool(ticker_re.fullmatch(tok)),
        "is_value": is_value_token(tok),
        "is_money_unit": is_money_unit(tok),
    }



In [3]:
# --------------------------------------------------
# 3) Construção de X (features) e y (labels)
# --------------------------------------------------
X, y = [], []
for sent in sentences:
    tokens = tokenize(sent)
    X.append([token2features(tokens, i) for i in range(len(tokens))])
    y.append(label_tokens(tokens))

# Exemplo rápido de inspeção
print("\nFrase tokenizada + rótulos:")
for t, lab in zip(tokenize(sentences[0]), y[0]):
    print(f"{t:>10s}  -->  {lab}")

print("\nFeatures do primeiro token da segunda frase:")
pprint(X[1][0])


Frase tokenizada + rótulos:
     Quero  -->  O
  investir  -->  O
         R  -->  O
         $  -->  B-VALOR
         7  -->  I-VALOR
       500  -->  I-VALOR
         ,  -->  O
        00  -->  B-VALOR
        em  -->  O
     PETR4  -->  B-ATIVO
    amanhã  -->  O
         .  -->  O

Features do primeiro token da segunda frase:
{'bias': 1.0,
 'has_digit': False,
 'is_digit': False,
 'is_money_unit': False,
 'is_ticker': False,
 'is_upper': False,
 'is_value': False,
 'lower': 'pretendo',
 'next_is_currency': False,
 'next_token': 'aplicar',
 'prev_is_currency': False,
 'prev_token': '__BOS__',
 'suffix2': 'do',
 'token': 'Pretendo'}


In [11]:
len(X[0][0])  # Exemplo de acesso a features do primeiro token da primeira frase

14

In [5]:
from sklearn_crfsuite import CRF

In [20]:
crf = CRF(algorithm="lbfgs", c1=0.1, c2=0.1, max_iterations=200,
          all_possible_transitions=True, verbose=True)
crf.fit(X, y)


loading training data to CRFsuite: 100%|██████████| 50/50 [00:00<00:00, 8674.88it/s]


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 902
Seconds required: 0.002

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.00  loss=565.69   active=894   feature_norm=1.00
Iter 2   time=0.00  loss=342.61   active=885   feature_norm=2.80
Iter 3   time=0.00  loss=204.32   active=881   feature_norm=3.99
Iter 4   time=0.00  loss=141.22   active=885   feature_norm=5.06
Iter 5   time=0.00  loss=93.45    active=870   feature_norm=6.93
Iter 6   time=0.00  loss=70.74    active=836   feature_norm=8.07
Iter 7   time=0.00  loss=58.16    active=824   feature_norm=8.88
Iter 8   time=0.00  loss=45.00    active=743   feature_norm=10.57
Iter 9   time=0.00  loss=42.15    active=704   feature_norm=10.96
Iter 10  time=

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    max_iterations=200, verbose=True)

In [17]:
# -------------------------------------------------
# 1) Frases novas para inferência
# -------------------------------------------------
sent = "Gostaria de investir R$ 55.000 em KNRI11 na próxima semana."

# -------------------------------------------------
# 2) Inferência
#    (pressupõe que você já tenha `tokenize`, `token2features`
#     e o objeto `crf` treinado em memória)
# -------------------------------------------------
tokens = tokenize(sent)
feats  = [token2features(tokens, i) for i in range(len(tokens))]
preds  = crf.predict_single(feats)        # sequência de rótulos
print(f"\nFrase: {sent}")
for tok, lab in zip(tokens, preds):
    print(f"{tok:>12s} -> {lab}")



Frase: Gostaria de investir R$ 55.000 em KNRI11 na próxima semana.
    Gostaria -> O
          de -> O
    investir -> O
           R -> O
           $ -> B-VALOR
          55 -> I-VALOR
           . -> O
         000 -> B-VALOR
          em -> O
      KNRI11 -> B-ATIVO
          na -> O
     próxima -> O
      semana -> O
           . -> O


In [18]:
preds

['O',
 'O',
 'O',
 'O',
 'B-VALOR',
 'I-VALOR',
 'O',
 'B-VALOR',
 'O',
 'B-ATIVO',
 'O',
 'O',
 'O',
 'O']

In [19]:
extract_entities(tokens, preds)

{'VALOR': ['$ 55', '000'], 'ATIVO': ['KNRI11']}